In [1]:
import openfermion as of

# FermiHubbard_Model

In [48]:
from openfermion.ops.operators import BosonOperator, FermionOperator
from openfermion.utils.indexing import down_index, up_index
from openfermion.hamiltonians.special_operators import number_operator


def fermi_hubbard(x_dimension,
                  y_dimension,
                  tunneling,
                  coulomb,
                  chemical_potential=0.,
                  magnetic_field=0.,
                  periodic=True,
                  spinless=False,
                  particle_hole_symmetry=False):
   
    if spinless:
        return _spinless_fermi_hubbard_model(x_dimension, y_dimension,
                                             tunneling, coulomb,
                                             chemical_potential, magnetic_field,
                                             periodic, particle_hole_symmetry)
    else:
        return _spinful_fermi_hubbard_model(x_dimension, y_dimension, tunneling,
                                            coulomb, chemical_potential,
                                            magnetic_field, periodic,
                                            particle_hole_symmetry)


def _spinful_fermi_hubbard_model(x_dimension, y_dimension, tunneling, coulomb,
                                 chemical_potential, magnetic_field, periodic,
                                 particle_hole_symmetry):

    # Initialize operator.
    n_sites = x_dimension * y_dimension
    n_spin_orbitals = 2 * n_sites
    hubbard_model = FermionOperator()

    # Loop through sites and add terms.
    for site in range(n_sites):

        # Get indices of right and bottom neighbors
        right_neighbor = _right_neighbor(site, x_dimension, y_dimension,
                                         periodic)
        bottom_neighbor = _bottom_neighbor(site, x_dimension, y_dimension,
                                           periodic)

        # Avoid double-counting edges when one of the dimensions is 2
        # and the system is periodic
        if x_dimension == 2 and periodic and site % 2 == 1:
            right_neighbor = None
        if y_dimension == 2 and periodic and site >= x_dimension:
            bottom_neighbor = None

        # Add hopping terms with neighbors to the right and bottom.
        if right_neighbor is not None:
            hubbard_model += _hopping_term(up_index(site),
                                           up_index(right_neighbor), -tunneling)
            hubbard_model += _hopping_term(down_index(site),
                                           down_index(right_neighbor),
                                           -tunneling)
        if bottom_neighbor is not None:
            hubbard_model += _hopping_term(up_index(site),
                                           up_index(bottom_neighbor),
                                           -tunneling)
            hubbard_model += _hopping_term(down_index(site),
                                           down_index(bottom_neighbor),
                                           -tunneling)

        # Add local pair Coulomb interaction terms.
        hubbard_model += _coulomb_interaction_term(n_spin_orbitals,
                                                   up_index(site),
                                                   down_index(site), coulomb,
                                                   particle_hole_symmetry)

        # Add chemical potential and magnetic field terms.
        hubbard_model += number_operator(n_spin_orbitals, up_index(site),
                                         -chemical_potential - magnetic_field)
        hubbard_model += number_operator(n_spin_orbitals, down_index(site),
                                         -chemical_potential + magnetic_field)

    return hubbard_model


def _spinless_fermi_hubbard_model(x_dimension, y_dimension, tunneling, coulomb,
                                  chemical_potential, magnetic_field, periodic,
                                  particle_hole_symmetry):

    # Initialize operator.
    n_sites = x_dimension * y_dimension
    hubbard_model = FermionOperator()

    # Loop through sites and add terms.
    for site in range(n_sites):

        # Get indices of right and bottom neighbors
        right_neighbor = _right_neighbor(site, x_dimension, y_dimension,
                                         periodic)
        bottom_neighbor = _bottom_neighbor(site, x_dimension, y_dimension,
                                           periodic)

        # Avoid double-counting edges when one of the dimensions is 2
        # and the system is periodic
        if x_dimension == 2 and periodic and site % 2 == 1:
            right_neighbor = None
        if y_dimension == 2 and periodic and site >= x_dimension:
            bottom_neighbor = None

        # Add terms that couple with neighbors to the right and bottom.
        if right_neighbor is not None:
            # Add hopping term
            hubbard_model += _hopping_term(site, right_neighbor, -tunneling)
            # Add local Coulomb interaction term
            hubbard_model += _coulomb_interaction_term(n_sites, site,
                                                       right_neighbor, coulomb,
                                                       particle_hole_symmetry)
        if bottom_neighbor is not None:
            # Add hopping term
            hubbard_model += _hopping_term(site, bottom_neighbor, -tunneling)
            # Add local Coulomb interaction term
            hubbard_model += _coulomb_interaction_term(n_sites, site,
                                                       bottom_neighbor, coulomb,
                                                       particle_hole_symmetry)

        # Add chemical potential. The magnetic field doesn't contribute.
        hubbard_model += number_operator(n_sites, site, -chemical_potential)

    return hubbard_model


def _hopping_term(i, j, coefficient, bosonic=False):
    op_class = BosonOperator if bosonic else FermionOperator
    hopping_term = op_class(((i, 1), (j, 0)), coefficient)
    hopping_term += op_class(((j, 1), (i, 0)), coefficient.conjugate())
    return hopping_term


def _coulomb_interaction_term(n_sites,
                              i,
                              j,
                              coefficient,
                              particle_hole_symmetry,
                              bosonic=False):
    op_class = BosonOperator if bosonic else FermionOperator
    number_operator_i = number_operator(n_sites, i, parity=2 * bosonic - 1)
    number_operator_j = number_operator(n_sites, j, parity=2 * bosonic - 1)
    if particle_hole_symmetry:
        number_operator_i -= op_class((), 0.5)
        number_operator_j -= op_class((), 0.5)
    return coefficient * number_operator_i * number_operator_j


def _right_neighbor(site, x_dimension, y_dimension, periodic):
    if x_dimension == 1:
        return None
    if (site + 1) % x_dimension == 0:
        if periodic:
            return site + 1 - x_dimension
        else:
            return None
    return site + 1


def _bottom_neighbor(site, x_dimension, y_dimension, periodic):
    if y_dimension == 1:
        return None
    if site + x_dimension + 1 > x_dimension * y_dimension:
        if periodic:
            return site + x_dimension - x_dimension * y_dimension
        else:
            return None
    return site + x_dimension

In [49]:
from openfermion.hamiltonians import fermi_hubbard
from openfermion.linalg import get_sparse_operator, get_ground_state
from openfermion.transforms import jordan_wigner
from openfermion.linalg import eigenspectrum

# Set model.
x_dimension = 2
y_dimension = 1
tunneling = -2.719
coulomb = 5.0
# Get fermion operator.
hubbard_model = fermi_hubbard(x_dimension,
                  y_dimension,
                  tunneling,
                  coulomb,
                  chemical_potential=0.,
                  magnetic_field=0.,
                  periodic=True,
                  spinless=False,
                  particle_hole_symmetry=False)
print(hubbard_model)

# Get qubit operator under Jordan-Wigner.
jw_hamiltonian = jordan_wigner(hubbard_model)
jw_hamiltonian.compress()
print('')
print(jw_hamiltonian)
jw_spectrum = eigenspectrum(jw_hamiltonian)
print("jw_eigenspectrum")
print(jw_spectrum)
# for i in range (len(jw_spectrum)-1):
#     exc_energy = jw_spectrum[i+1]-jw_spectrum[i]
#     if exc_energy > 1.0 :
#         print ("excitation energy",exc_energy)
#         break
#     break
print('')


# Get scipy.sparse.csc representation.
sparse_operator = get_sparse_operator(hubbard_model)
print('')
print(sparse_operator)
print('\nEnergy of the model is {} in units of T and J.'.format(
    get_ground_state(sparse_operator)[0]))

5.0 [0^ 0 1^ 1] +
2.719 [0^ 2] +
2.719 [1^ 3] +
2.719 [2^ 0] +
5.0 [2^ 2 3^ 3] +
2.719 [3^ 1]

2.5 [] +
1.3595 [X0 Z1 X2] +
1.3595 [Y0 Z1 Y2] +
-1.25 [Z0] +
1.25 [Z0 Z1] +
1.3595 [X1 Z2 X3] +
1.3595 [Y1 Z2 Y3] +
-1.25 [Z1] +
-1.25 [Z2] +
1.25 [Z2 Z3] +
-1.25 [Z3]
jw_eigenspectrum
[-3.48513525e+00 -2.71900000e+00 -2.71900000e+00 -9.27437506e-16
  0.00000000e+00  2.77555756e-16  5.47196519e-16  2.28100000e+00
  2.28100000e+00  2.71900000e+00  2.71900000e+00  5.00000000e+00
  7.71900000e+00  7.71900000e+00  8.48513525e+00  1.00000000e+01]


  (4, 1)	(2.719+0j)
  (8, 2)	(2.719+0j)
  (3, 3)	(5+0j)
  (6, 3)	(-2.719+0j)
  (9, 3)	(2.719+0j)
  (1, 4)	(2.719+0j)
  (3, 6)	(-2.719+0j)
  (12, 6)	(-2.719+0j)
  (7, 7)	(5+0j)
  (13, 7)	(-2.719+0j)
  (2, 8)	(2.719+0j)
  (3, 9)	(2.719+0j)
  (12, 9)	(2.719+0j)
  (11, 11)	(5+0j)
  (14, 11)	(-2.719+0j)
  (6, 12)	(-2.719+0j)
  (9, 12)	(2.719+0j)
  (12, 12)	(5+0j)
  (7, 13)	(-2.719+0j)
  (13, 13)	(5+0j)
  (11, 14)	(-2.719+0j)
  (14, 14)	(5+0j)
  (15, 15)	(10

In [4]:
from openfermion.linalg import get_sparse_operator
import numpy
import sys
from sympy import *
from numpy import linalg as LA
numpy.set_printoptions(threshold=sys.maxsize)
hamiltonian_matrix = get_sparse_operator(hubbard_model)
#print(hamiltonian_matrix)

dense_matrix = hamiltonian_matrix.toarray()
#print(dense_matrix)

P,E = LA.eig((dense_matrix))
print(P)
print("")
#ham_spectrum = eigenspectrum(hamiltonian_matrix)

[-3.88729884+0.j  7.60729884+0.j  3.72      +0.j -2.719     +0.j
  2.719     +0.j -2.719     +0.j  2.719     +0.j  6.439     +0.j
  1.001     +0.j  0.        +0.j  6.439     +0.j  1.001     +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  7.44      +0.j]



# PPP_Model

In [50]:
from openfermion.ops.operators import BosonOperator, FermionOperator
from openfermion.utils.indexing import down_index, up_index
from openfermion.hamiltonians.special_operators import number_operator


def fermi_ppp(z,
                  unit_dist,
                  x_dimension,
                  y_dimension,
                  tunneling,
                  coulomb,
                  chemical_potential=0.,
                  magnetic_field=0.,
                  periodic=True,
                  spinless=False,
                  particle_hole_symmetry=False):
   
    if spinless:
        return _spinless_fermi_hubbard_model(x_dimension, y_dimension,
                                             tunneling, coulomb,
                                             chemical_potential, magnetic_field,
                                             periodic, particle_hole_symmetry,z,unit_dist)
    else:
        return _spinful_fermi_hubbard_model(x_dimension, y_dimension, tunneling,
                                            coulomb, chemical_potential,
                                            magnetic_field, periodic,
                                            particle_hole_symmetry,z,unit_dist)


def _spinful_fermi_hubbard_model(x_dimension, y_dimension, tunneling, coulomb,
                                 chemical_potential, magnetic_field, periodic,
                                 particle_hole_symmetry,z,unit_dist):

    # Initialize operator.
    n_sites = x_dimension * y_dimension
    n_spin_orbitals = 2 * n_sites
    hubbard_model = FermionOperator()

    # Loop through sites and add terms.
    for site in range(n_sites):

        # Get indices of right and bottom neighbors
        right_neighbor = _right_neighbor(site, x_dimension, y_dimension,
                                         periodic)
        bottom_neighbor = _bottom_neighbor(site, x_dimension, y_dimension,
                                           periodic)

        # Avoid double-counting edges when one of the dimensions is 2
        # and the system is periodic
        if x_dimension == 2 and periodic and site % 2 == 1:
            right_neighbor = None
        if y_dimension == 2 and periodic and site >= x_dimension:
            bottom_neighbor = None

        # Add hopping terms with neighbors to the right and bottom.
        if right_neighbor is not None:
            hubbard_model += _hopping_term(up_index(site),
                                           up_index(right_neighbor), -tunneling)
            hubbard_model += _hopping_term(down_index(site),
                                           down_index(right_neighbor),
                                           -tunneling)
        if bottom_neighbor is not None:
            hubbard_model += _hopping_term(up_index(site),
                                           up_index(bottom_neighbor),
                                           -tunneling)
            hubbard_model += _hopping_term(down_index(site),
                                           down_index(bottom_neighbor),
                                           -tunneling)

        # Add local pair Coulomb interaction terms.
        hubbard_model += _coulomb_interaction_term(n_spin_orbitals,
                                                   up_index(site),
                                                   down_index(site), coulomb,
                                                   particle_hole_symmetry)

        # Add chemical potential and magnetic field terms.
        hubbard_model += number_operator(n_spin_orbitals, up_index(site),
                                         -chemical_potential - magnetic_field)
        hubbard_model += number_operator(n_spin_orbitals, down_index(site),
                                         -chemical_potential + magnetic_field)
        
        # Add intersite_interaction terms.
        hubbard_model += _intersite_interaction_term(unit_dist, coulomb, x_dimension, z)
       
    return hubbard_model


def _spinless_fermi_hubbard_model(x_dimension, y_dimension, tunneling, coulomb,
                                  chemical_potential, magnetic_field, periodic,
                                  particle_hole_symmetry,z,unit_dist):

    # Initialize operator.
    n_sites = x_dimension * y_dimension
    hubbard_model = FermionOperator()

    # Loop through sites and add terms.
    for site in range(n_sites):

        # Get indices of right and bottom neighbors
        right_neighbor = _right_neighbor(site, x_dimension, y_dimension,
                                         periodic)
        bottom_neighbor = _bottom_neighbor(site, x_dimension, y_dimension,
                                           periodic)

        # Avoid double-counting edges when one of the dimensions is 2
        # and the system is periodic
        if x_dimension == 2 and periodic and site % 2 == 1:
            right_neighbor = None
        if y_dimension == 2 and periodic and site >= x_dimension:
            bottom_neighbor = None

        # Add terms that couple with neighbors to the right and bottom.
        if right_neighbor is not None:
            # Add hopping term
            hubbard_model += _hopping_term(site, right_neighbor, -tunneling)
            # Add local Coulomb interaction term
            hubbard_model += _coulomb_interaction_term(n_sites, site,
                                                       right_neighbor, coulomb,
                                                       particle_hole_symmetry)
            # Add intersite_interaction terms.
            hubbard_model += _intersite_interaction_term(unit_dist, coulomb, x_dimension, z)
        if bottom_neighbor is not None:
            # Add hopping term
            hubbard_model += _hopping_term(site, bottom_neighbor, -tunneling)
            # Add local Coulomb interaction term
            hubbard_model += _coulomb_interaction_term(n_sites, site,
                                                       bottom_neighbor, coulomb,
                                                       particle_hole_symmetry)
            # Add intersite_interaction terms.
            hubbard_model += _intersite_interaction_term(unit_dist, coulomb, x_dimension, z)

        # Add chemical potential. The magnetic field doesn't contribute.
        hubbard_model += number_operator(n_sites, site, -chemical_potential)

    return hubbard_model


def _hopping_term(i, j, coefficient, bosonic=False):
    op_class = BosonOperator if bosonic else FermionOperator
    hopping_term = op_class(((i, 1), (j, 0)), coefficient)
    hopping_term += op_class(((j, 1), (i, 0)), coefficient.conjugate())
    return hopping_term


def _coulomb_interaction_term(n_sites,
                              i,
                              j,
                              coefficient,
                              particle_hole_symmetry,
                              bosonic=False):
    op_class = BosonOperator if bosonic else FermionOperator
    number_operator_i = number_operator(n_sites, i, parity=2 * bosonic - 1)
    number_operator_j = number_operator(n_sites, j, parity=2 * bosonic - 1)
    if particle_hole_symmetry:
        number_operator_i -= op_class((), 0.5)
        number_operator_j -= op_class((), 0.5)
    return coefficient * number_operator_i * number_operator_j

def _intersite_interaction_term(unit_dist, U, irange, z):
    intersite = None
    n_sites = x_dimension*y_dimension
    for i in range(1, irange+1):
        for j in range(i, n_sites+1):
            if i != j:
                gamma = _mataga_nishimoto(unit_dist, U, i, j)
                ni, nj = _get_n(z,i, j)
                intersite = gamma*ni*nj
    return intersite 

def _mataga_nishimoto(unit_dist, U, i, j):
    rij = ((j-i)*1.4)/unit_dist
    gamma = 1/((1/U)+rij)
    return gamma


def _get_n(z,i, j):
    n_sites = x_dimension*y_dimension
    op_class = FermionOperator
    n_i = number_operator(n_sites, i)
    n_j = number_operator(n_sites, j)
    print(n_i)
    return (z-n_i, z-n_j)


def _right_neighbor(site, x_dimension, y_dimension, periodic):
    if x_dimension == 1:
        return None
    if (site + 1) % x_dimension == 0:
        if periodic:
            return site + 1 - x_dimension
        else:
            return None
    return site + 1


def _bottom_neighbor(site, x_dimension, y_dimension, periodic):
    if y_dimension == 1:
        return None
    if site + x_dimension + 1 > x_dimension * y_dimension:
        if periodic:
            return site + x_dimension - x_dimension * y_dimension
        else:
            return None
    return site + x_dimension

In [72]:
"""Define the Hamiltonian."""
# Parameters.
nsites = 4
U = 8.325
J = -2.719
z = 1
x_dimension = nsites
y_dimension = 1
# unit_dist = 0.9107e-8
unit_dist = 1.27
ppp = fermi_ppp(z,
                  unit_dist,
                  x_dimension,
                  y_dimension,
                  -J,
                  U,
                  chemical_potential=0.,
                  magnetic_field=0.,
                  periodic=True,
                  spinless=False,
                  particle_hole_symmetry=False)
print(ppp)

1.0 [1^ 1]
1.0 [1^ 1]
1.0 [1^ 1]
1.0 [2^ 2]
1.0 [2^ 2]
1.0 [3^ 3]
1.0 [1^ 1]
1.0 [1^ 1]
1.0 [1^ 1]
1.0 [2^ 2]
1.0 [2^ 2]
1.0 [3^ 3]
1.0 [1^ 1]
1.0 [1^ 1]
1.0 [1^ 1]
1.0 [2^ 2]
1.0 [2^ 2]
1.0 [3^ 3]
1.0 [1^ 1]
1.0 [1^ 1]
1.0 [1^ 1]
1.0 [2^ 2]
1.0 [2^ 2]
1.0 [3^ 3]
3.272030947775629 [] +
8.325 [0^ 0 1^ 1] +
-2.719 [0^ 2] +
-2.719 [0^ 6] +
-2.719 [1^ 3] +
-2.719 [1^ 7] +
-2.719 [2^ 0] +
8.325 [2^ 2 3^ 3] +
-2.719 [2^ 4] +
-2.719 [3^ 1] +
-3.272030947775629 [3^ 3] +
3.272030947775629 [3^ 3 4^ 4] +
-2.719 [3^ 5] +
-2.719 [4^ 2] +
-3.272030947775629 [4^ 4] +
8.325 [4^ 4 5^ 5] +
-2.719 [4^ 6] +
-2.719 [5^ 3] +
-2.719 [5^ 7] +
-2.719 [6^ 0] +
-2.719 [6^ 4] +
8.325 [6^ 6 7^ 7] +
-2.719 [7^ 1] +
-2.719 [7^ 5]


In [73]:
from openfermion.linalg import get_sparse_operator
import numpy
import sys
from sympy import *
from numpy import linalg as LA
numpy.set_printoptions(threshold=sys.maxsize)
hamiltonian_matrix = get_sparse_operator(ppp)
#print(hamiltonian_matrix)

dense_matrix = hamiltonian_matrix.toarray()
#print(dense_matrix)

P,E = LA.eig((dense_matrix))
#print(P)
print("")
#ham_spectrum = eigenspectrum(hamiltonian_matrix)

In [74]:
from openfermion.linalg import get_sparse_operator, get_ground_state
from openfermion.transforms import jordan_wigner
from openfermion.linalg import eigenspectrum

# Get qubit operator under Jordan-Wigner.
jw_hamiltonian = jordan_wigner(ppp)
jw_hamiltonian.compress()
# print("jw_hamiltonian")
#print(jw_hamiltonian)
print('')

jw_spectrum = eigenspectrum(jw_hamiltonian)
# print("jw_eigenspectrum")
print(jw_spectrum)
print('')

qubit_matrix = get_sparse_operator(jw_hamiltonian)
# print("Qubit Matrix")
# print(qubit_matrix)

for i in range (len(jw_spectrum)-1):
    exc_energy = jw_spectrum[i+1]-jw_spectrum[i]
    if exc_energy > 0.1 :
        print ("excitation energy",exc_energy)
        break
    break


print('\nEnergy of the model is {} .'.format(
   get_ground_state(qubit_matrix)[0]))


[-8.07770747e+00 -7.57931182e+00 -7.57931182e+00 -6.40856321e+00
 -6.40856321e+00 -6.18554818e+00 -5.45367090e+00 -5.45367090e+00
 -4.86032612e+00 -4.86032612e+00 -4.86032612e+00 -4.86032612e+00
 -4.61498243e+00 -4.61498243e+00 -4.36817005e+00 -4.18364343e+00
 -3.46861804e+00 -3.41678187e+00 -3.35924884e+00 -3.35924884e+00
 -3.35924884e+00 -3.35924884e+00 -3.02872110e+00 -3.02872110e+00
 -2.62753099e+00 -2.62753099e+00 -2.57057519e+00 -2.53817345e+00
 -2.53817345e+00 -1.66317708e+00 -1.66317708e+00 -1.07665520e+00
 -9.66732949e-01 -7.46550151e-01 -6.85661635e-01 -2.48665189e-01
 -2.48665189e-01 -7.50667729e-15 -1.78018352e-16  1.02938589e-15
  4.09213623e-15  4.60024023e-01  4.60024023e-01  6.32906541e-01
  6.32906541e-01  7.89714466e-01  8.75369242e-01  8.75369242e-01
  1.20079391e+00  1.20079391e+00  1.25437477e+00  1.25437477e+00
  1.38803894e+00  1.50107728e+00  1.50107728e+00  1.50107728e+00
  1.50107728e+00  1.62828942e+00  1.62828942e+00  1.69468598e+00
  1.77095367e+00  1.7709